# 🚀 產品推薦系統 - 超簡化版

這個版本直接使用協同過濾，不需要訓練複雜模型。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ldsAS/product-recommendation-system/blob/main/notebooks/colab_simple_demo.ipynb)

In [ ]:
# 安裝依賴
!pip install -q pandas numpy scikit-learn pydantic pydantic-settings

print("✅ 套件安裝完成！")

In [ ]:
# 克隆專案
!git clone https://github.com/ldsAS/product-recommendation-system.git
%cd product-recommendation-system

import os
import sys
sys.path.insert(0, '/content/product-recommendation-system')

print("✅ 專案設置完成！")

In [ ]:
# 生成簡單的測試資料
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

# 會員資料
members_data = [
    {'會員編號': 'CU000001', '電話': '0937024682', '總消費金額': 17400, '累積紅利': 500},
    {'會員編號': 'CU000002', '電話': '0912345678', '總消費金額': 25600, '累積紅利': 800},
    {'會員編號': 'CU000003', '電話': '0923456789', '總消費金額': 12300, '累積紅利': 300},
]

# 產品列表
products = ['30001', '30002', '30003', '30004', '30005', '30006', '30007', '30008', '30009', '30010']

# 購買歷史（會員-產品矩陣）
purchase_history = {
    'CU000001': ['30001', '30002', '30005'],
    'CU000002': ['30001', '30003', '30004'],
    'CU000003': ['30002', '30005', '30006'],
}

print("✅ 測試資料已創建")
print(f"會員數: {len(members_data)}")
print(f"產品數: {len(products)}")

In [ ]:
# 簡單的協同過濾推薦
from collections import Counter

def get_recommendations(member_id, purchase_history, top_k=5):
    """
    基於協同過濾的簡單推薦
    找出購買相似產品的其他會員，推薦他們買過但目標會員沒買過的產品
    """
    if member_id not in purchase_history:
        return []
    
    # 目標會員購買的產品
    user_products = set(purchase_history[member_id])
    
    # 找出相似會員（有共同購買產品的會員）
    similar_users = []
    for other_user, other_products in purchase_history.items():
        if other_user != member_id:
            # 計算共同產品數
            common = len(user_products & set(other_products))
            if common > 0:
                similar_users.append((other_user, common))
    
    # 收集推薦產品
    recommendations = Counter()
    for other_user, similarity in similar_users:
        # 找出其他會員買過但目標會員沒買過的產品
        new_products = set(purchase_history[other_user]) - user_products
        for product in new_products:
            recommendations[product] += similarity
    
    # 返回 Top K
    return [product for product, score in recommendations.most_common(top_k)]

# 測試推薦
test_member = 'CU000001'
recommendations = get_recommendations(test_member, purchase_history)

print(f"\n🎯 為會員 {test_member} 生成推薦")
print(f"已購買: {purchase_history[test_member]}")
print(f"\n✨ 推薦產品:")
for i, product in enumerate(recommendations, 1):
    print(f"{i}. 產品 {product}")

if not recommendations:
    print("（暫無推薦，可能是資料太少）")

In [ ]:
# 為所有會員生成推薦
print("\n" + "=" * 60)
print("📊 所有會員的推薦結果")
print("=" * 60)

for member in members_data:
    member_id = member['會員編號']
    recs = get_recommendations(member_id, purchase_history, top_k=3)
    
    print(f"\n會員 {member_id}:")
    print(f"  消費金額: ${member['總消費金額']:,}")
    print(f"  已購買: {purchase_history.get(member_id, [])}")
    print(f"  推薦: {recs if recs else '暫無推薦'}")

In [ ]:
# 視覺化購買矩陣
import matplotlib.pyplot as plt
import seaborn as sns

# 創建會員-產品矩陣
matrix_data = []
member_ids = list(purchase_history.keys())
all_products = sorted(set(p for products in purchase_history.values() for p in products))

for member_id in member_ids:
    row = [1 if product in purchase_history[member_id] else 0 for product in all_products]
    matrix_data.append(row)

# 繪製熱力圖
plt.figure(figsize=(10, 4))
sns.heatmap(matrix_data, 
            xticklabels=all_products, 
            yticklabels=member_ids,
            cmap='YlOrRd',
            cbar_kws={'label': 'Purchased'},
            linewidths=0.5)
plt.title('Member-Product Purchase Matrix')
plt.xlabel('Product ID')
plt.ylabel('Member ID')
plt.tight_layout()
plt.show()

print("\n✅ 視覺化完成！")

## 🎉 完成！

這個簡化版展示了推薦系統的核心概念：
- 協同過濾：找出相似會員
- 推薦邏輯：推薦相似會員買過的產品
- 視覺化：展示購買模式

### 完整版功能

完整的系統還包括：
- 機器學習模型（LightGBM/XGBoost）
- 特徵工程
- 模型評估
- RESTful API
- 推薦理由生成

查看完整文檔：[GitHub](https://github.com/ldsAS/product-recommendation-system)